In [13]:
import numpy as np
from scipy.signal import spectrogram
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import os
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy




import mne

import numpy as np


def getXYdata(data):
    from sklearn.preprocessing import LabelEncoder
    yData = []
    X = data.get_data()
    contador = 0
    start_t = data.annotations.onset
    for i in range(0,X.shape[1]):
        time = i/160
        if contador < data.annotations.description.shape[0]:
            if time >= start_t[contador]:
                contador = contador + 1
        yData.append(data.annotations.description[contador-1])
    yData = np.array(yData)
    le = LabelEncoder()
    le.fit(yData)
    yData = le.transform(yData)
    return X.T,yData

In [14]:
def splitTimeSeries(Xdata, ydata, n_steps,stride):

    for i in range(n_steps, len(Xdata) - 1,stride):
        X = Xdata[i - n_steps:i]
        y = ydata[i+1]

        yield X, y
    


def generator_function(basePath,n_steps, stride):
    for individual in range(1, 110):  # valid individuals are 1 to 109
        for exam in range(3, 15):  # valid tests for an individual are 3 to 14
            file = basePath + f"S{str(individual).zfill(3)}/S{str(individual).zfill(3)}R{str(exam).zfill(2)}.edf"
            data = mne.io.read_raw_edf(file)
            Xdata, yData = getXYdata(data)
            yield from splitTimeSeries(Xdata, yData, n_steps, stride)

def create_tf_dataset(basePath, n_steps, stride, batch_size):
    generator = lambda: generator_function(basePath, n_steps, stride)
    dataset = tf.data.Dataset.from_generator(generator, output_signature=(
        tf.TensorSpec(shape=(n_steps, 64), dtype=tf.float64),
        tf.TensorSpec(shape=(), dtype=tf.int64)
    ))
    dataset = dataset.batch(batch_size)
    return dataset

# Example usage
basePath = "physionet.org/files/eegmmidb/1.0.0/"
n_steps = 700
stride = 1
batch_size = 512

# Create the full dataset
full_dataset = create_tf_dataset(basePath, n_steps, stride, batch_size)


In [15]:



features = 64
model = Sequential()
model.add(LSTM(units=64, activation="tanh", return_sequences=False, input_shape=(n_steps, features)))
model.add(Dense(units=3, activation="softmax"))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # Use 'accuracy' as a metric

# More concise training loop using model.fit
epochs = 100
result = model.fit(full_dataset)

# Plotting code
plt.title("training results")
plt.plot(result.history['loss'], label='train_loss')
plt.plot(result.history['accuracy'], label='train_accuracy')
plt.legend()
plt.show()


Extracting EDF parameters from /home/andre/Desktop/MachineLearning/Movement_EEG/physionet.org/files/eegmmidb/1.0.0/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
     37/Unknown - 7s 148ms/step - loss: 1.0881 - accuracy: 0.4235Extracting EDF parameters from /home/andre/Desktop/MachineLearning/Movement_EEG/physionet.org/files/eegmmidb/1.0.0/S001/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
     74/Unknown - 12s 149ms/step - loss: 1.0681 - accuracy: 0.4600Extracting EDF parameters from /home/andre/Desktop/MachineLearning/Movement_EEG/physionet.org/files/eegmmidb/1.0.0/S001/S001R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
    112/Unknown - 18s 147ms/step - loss: 1.0639 - accuracy: 0.4680Extracting EDF parameters from /home/andre/Desktop/MachineLearning/Movement_EEG/physionet.org/files/eegmmidb/1.0.0/S001/S001R06.edf...
EDF file detec